In [ ]:
!pip install ultralytics tensorflow mediapipe opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 32.2 MB/s eta 0:00:00


In [ ]:
#functions
from ultralytics import YOLO
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import datetime

detector = YOLO('yolov8n.pt')
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
gender_model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)
mp_drawing = mp.solutions.drawing_utils

def process_frame(frame):
    results = detector(frame)
    boxes = results[0].boxes.xyxy
    gender_counts = {"Male": 0, "Female": 0}
    genders = []

    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        person_crop = frame[y1:y2, x1:x2]
        person_crop_resized = cv2.resize(person_crop, (224, 224))
        input_tensor = np.expand_dims(person_crop_resized, axis=0) / 255.0
        gender_prediction = gender_model.predict(input_tensor)
        gender = "Male" if gender_prediction[0][0] > 0.5 else "Female"
        gender_counts[gender] += 1
        genders.append((x1, y1, x2, y2, gender))

    current_hour = datetime.datetime.now().hour
    if current_hour >= 19 and gender_counts["Female"] == 1 and gender_counts["Male"] == 0:
        print("Alert: Lone woman detected at night!")

    if gender_counts["Female"] == 1 and gender_counts["Male"] >= 3:
        print("Alert: Woman surrounded by men!")

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hand_results = hands.process(rgb_frame)
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            wrist_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
            index_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
            if wrist_y < index_tip_y:
                print("SOS Gesture Detected!")

    for (x1, y1, x2, y2, gender) in genders:
        color = (0, 255, 0) if gender == "Female" else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, gender, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    return frame

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 44.4MB/s]


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
#processing video
def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = process_frame(frame)
        out.write(processed_frame)

    cap.release()
    out.release()
    print("Video processing complete. Saved at:", output_path)

In [ ]:
from google.colab import files

uploaded = files.upload()
input_video = list(uploaded.keys())[0]
output_video = "processed_output.mp4"

process_video(input_video, output_video)

files.download(output_video)

Saving imdai video 2.mp4 to imdai video 2.mp4

0: 256x640 2 persons, 1 car, 108.3ms
Speed: 4.9ms preprocess, 108.3ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step

0: 256x640 2 persons, 1 car, 116.2ms
Speed: 5.1ms preprocess, 116.2ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step

0: 256x640 2 persons, 1 car, 101.1ms
Speed: 3.0ms preprocess, 101.1ms inference, 1.1ms postprocess per image at shape (1, 3, 256, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step

0: 256x640 2 persons, 1 car, 119.4ms
Speed: 3.7ms preprocess, 119.4ms inference, 1.1ms postprocess per image at shape (1, 3, 256, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58

KeyboardInterrupt: 